# Text Generation

Meta Llama-3-8B_Instruct 모델의 Chat Template를 이용하여 generate_response() 함수를 정의하고 NLP Task 들에 대해 테스트해 보겠습니다.

## 0. Setup

In [1]:
import os
os.environ['HF_HOME'] = 'D:/HF/cache'
os.environ['HF_DATASETS'] = 'D:/HF/datasets'
os.environ['HF_HUB_DISABLE_SYMLINKS_WARNING'] = "1"

In [2]:
# !pip install transformers==4.51.1
# !pip install datasets==3.2.0

In [3]:
from IPython.display import display
from tqdm.notebook import tqdm as notebook_tqdm

In [4]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["FSDP_CPU_RAM_EFFICIENT_LOADING"] = "false"

In [6]:
!nvidia-smi

Mon Aug 18 14:57:41 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 553.62                 Driver Version: 553.62         CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A10-24Q               WDDM  |   00000002:00:00.0 Off |                    0 |
| N/A    0C    P8             N/A /  N/A  |    1087MiB /  24512MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

## 1. Llama-3-8B-Instruct Model

Meta Llama-3-8B-Instruct 모델을 가져오도록 하겠습니다.  
모델을 가져오기 위해서는 **Hugging Face Access Token**이 필요하며, Llama 모델 사용에 대한 사전 신청이 필요합니다.  
(https://huggingface.co/meta-llama/Meta-Llama-3-8B-Instruct)

In [7]:
"""
from huggingface_hub import notebook_login
notebook_login()
"""

'\nfrom huggingface_hub import notebook_login\nnotebook_login()\n'

모델 접근 권한이 어려운 경우 아래의 로컬 디렉토리(Group Volume)에 있는 모델을 활용하시면 됩니다.

In [8]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# model_ckpt = "meta-llama/Meta-Llama-3-8B-Instruct"
model_ckpt = "//swschoolavdazfiles002.file.core.windows.net/aias-language/Model/Meta-Llama-3-8B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
tokenizer.pad_token_id = tokenizer.eos_token_id

model = AutoModelForCausalLM.from_pretrained(
    model_ckpt,
    torch_dtype=torch.bfloat16, # float16
    device_map="auto",
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [10]:
def generate_response(system_message, user_message):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message},
    ]

    input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)

    attention_mask = (input_ids != tokenizer.pad_token_id).long()

    terminators = [
        tokenizer.eos_token_id,
        tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    outputs = model.generate(
        input_ids,
        max_new_tokens=512,
        eos_token_id=terminators,
        attention_mask=attention_mask,
        pad_token_id=tokenizer.eos_token_id,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
    )

    response = outputs[0][input_ids.shape[-1]:]

    return tokenizer.decode(response, skip_special_tokens=True)

## 2. Llama-3-8B-Instruct Model Test (ChatBot, Translation, Coding, etc.)

**generate_response( )** 함수를 이용하여 다양한 NLP Task 들에 대해 테스트해 보겠습니다.

In [11]:
# [실습] 다음 코드를 완성하세요!!
# 다양한 NLP Task를 테스트해 보세요.
response = generate_response(system_message="",
                             user_message="여름을 주제로 시를 작성해줘.")

print(response)

Here's a poem about summer:

Summer's warmth descends upon me
A gentle breeze that whispers sweet melody
The sun shines bright, a golden glow
As petals bloom, and all is aglow

The scent of ripened fruit fills the air
As children's laughter echoes, without a care
The sound of splashing water, a joyful sound
As friends and family gather, without a bound

The stars appear, a twinkling show
As night falls soft, and the world slows down to grow
The fireflies dance, a twinkling light
As the world is bathed, in a warm and peaceful night

Summer's warmth, a time of rest and play
A season of adventure, every single day
A time to make memories, that will never fade
A season of joy, that will forever be displayed

I hope you enjoy it! 😊


In [12]:
# [실습] 다음 코드를 완성하세요!!
# 다양한 NLP Task를 테스트해 보세요.
response = generate_response(system_message="",
                             user_message="다음 문장을 한국어로 번역해줘, Time flies like an arrow.")

print(response)

시간은 화살과 같이 날아간다.

(Note: This is a common English idiom that means "time passes quickly" or "time goes by fast". The translation I provided is a literal translation, but it's worth noting that the idiomatic expression may not be as widely understood or used in Korean culture.)


In [13]:
# [실습] 다음 코드를 완성하세요!!
# 다양한 NLP Task를 테스트해 보세요.
response = generate_response(system_message="",
                             user_message="퀵소트 파이썬 알고리즘 작성해줘.")

print(response)

QuickSort는 재귀적으로 정렬하는 알고리즘입니다. 이 알고리즘은 다음과 같은 방법으로 작동합니다.

1. 배열의 길이가 1보다 작거나 같으면 이미 정렬된 배열이므로 반환합니다.
2. 배열의 중간 요소를 선택합니다. 이 요소를 피벗(pivot)이라고 합니다.
3. 피벗보다 작은 요소들은 왼쪽에, 피벗보다 큰 요소들은 오른쪽에 배치합니다.
4. 왼쪽과 오른쪽의 배열을 각각 재귀적으로 정렬합니다.
5. 정렬된 왼쪽과 오른쪽 배열을 결합하여 최종적으로 정렬된 배열을 얻습니다.

다음은 Python 코드입니다.
```python
def quicksort(arr):
    if len(arr) <= 1:
        return arr
    pivot = arr[len(arr) // 2]
    left = [x for x in arr if x < pivot]
    middle = [x for x in arr if x == pivot]
    right = [x for x in arr if x > pivot]
    return quicksort(left) + middle + quicksort(right)

# 테스트
arr = [3, 6, 1, 8, 2, 4]
print(quicksort(arr))  # [1, 2, 3, 4, 6, 8]
```
위 코드에서는 `quicksort` 함수를 정의합니다. 이 함수는 배열 `arr`를 정렬합니다. 이 함수는 재귀적으로 작동하므로, 배열의 길이가 1보다 작거나 같으면 이미 정렬된 배열이므로 반환합니다.

그리고 피벗을 선택하고, 피벗보다 작은 요소들은 왼쪽에, 피벗보다 큰 요소들은 오른쪽에 배치합니다. 그리고 왼쪽과 오른쪽의 배열을 각각 재귀적으로 정렬합니다. 그리고 정렬된 왼쪽과 오른쪽 배열을 결합하여 최종적으로 정렬된 배열을 얻습니다.

테스트 코드에서는 `[3, 6, 1, 8, 2, 4]` 배열을 정렬하고, 출력합니다. 출력된 배열은 `[1, 2, 3, 4, 6, 8]`입니다.
